### <a id="cont">Motive

Violence is last resort of the Incompetent... 
                               - Issac Asimov, Russian Born American Writer. 

Logic and mathematics are the languages this universe is programmed. When these two language fails, then Violence is taken as a last resort. Logic is 4 Dimensional. It has got Time as the 4th Dimension, where the leaders have to decide and act logically to avoid the last resort. They should avoid breaking the LOGIC. 

This dataset, and the analysis of it has to be a GLUE that makes the broken logic a whole. That is the way to resolve this conflict and establish peace. The analysis below shows 

1) [Where is the War happening?](#vis_1)

2) [How is the distribution of all equipments on last day?](#vis_2)

3) [What has changed on the last day?](#vis_3)

4) [How many machines are used in the War?](#vis_4)
    
5) [How many humans are used in the War?](#vis_5)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import plotly.express as px
import plotly.graph_objects as go
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ukraine-russian-war/russia_losses_equipment.csv
/kaggle/input/2022-ukraine-russian-war/russia_losses_personnel.csv


In [2]:
equipment = pd.read_csv("/kaggle/input/2022-ukraine-russian-war/russia_losses_equipment.csv")
personnel = pd.read_csv("/kaggle/input/2022-ukraine-russian-war/russia_losses_personnel.csv")

In [3]:
equipment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date                   6 non-null      object
 1   day                    6 non-null      int64 
 2   aircraft               6 non-null      int64 
 3   helicopter             6 non-null      int64 
 4   tank                   6 non-null      int64 
 5   APC                    6 non-null      int64 
 6   field artillery        6 non-null      int64 
 7   BUK                    6 non-null      int64 
 8   MRL                    6 non-null      int64 
 9   military auto          6 non-null      int64 
 10  fuel tank              6 non-null      int64 
 11  drone                  6 non-null      int64 
 12  naval ship             6 non-null      int64 
 13  anti-aircraft warfare  6 non-null      int64 
dtypes: int64(13), object(1)
memory usage: 800.0+ bytes


In [4]:
personnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       6 non-null      object 
 1   day        6 non-null      int64  
 2   personnel  6 non-null      int64  
 3   POW        1 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 320.0+ bytes


In [5]:
#Joining the data frames on the days columns to streamline the visualisation
manNmachine = pd.merge(left=equipment,right=personnel,left_on='day',right_on='day',how='left')

manNmachine.drop('date_y',axis=1,inplace=True)
manNmachine.head()

,date_x,day,aircraft,helicopter,tank,APC,field artillery,BUK,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,personnel,POW
0,2022-02-26,3,27,26,146,706,49,1,4,30,60,2,2,0,4300,NaN
1,2022-02-28,5,29,29,150,816,74,1,21,291,60,3,2,5,5300,NaN
2,2022-03-01,6,29,29,198,846,77,1,24,305,60,3,2,7,5710,200.0
3,2022-03-02,7,30,31,211,862,85,1,40,355,60,3,2,9,5840,NaN
4,2022-03-03,8,30,31,217,900,90,1,42,374,60,3,2,11,9000,NaN


In [6]:
manNmachine['totalEquipment'] = manNmachine.iloc[:,3:-2].sum(axis=1)

POW - Prisoner of War,

MRL - Multiple Rocket Launcher,

BUK - Buk Missile System,

APC - Armored Personnel Carrier,

drone: UAV - Unmanned Aerial Vehicle, RPA - Remotely Piloted Vehicle.

### Important. Each new record is accumulated data from previous days.

### What is getting added to the War?

In [7]:
def build_bar(dataset,column):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=dataset.index,y=dataset[column],name=column))
    fig.update_layout(height=800,title_text='How many Russian '+ column + ' are spotted everyday?')
    return fig

### <a id="vis_4">How many machines are used in the War? 

In [8]:
build_bar(manNmachine,'totalEquipment')

[Back to Contents](#cont)

### <a id="vis_5"> How many Army, Navy and Air force men and women are involved

In [9]:
build_bar(manNmachine,'personnel')

[Back to Contents](#cont)

In [10]:
build_bar(manNmachine,'aircraft')

[Back to Contents](#cont)

### <a id="vis_3"> What has changed on the last day?

In [11]:
#The last change
manmachine = manNmachine.columns[3:]
Last_change = manNmachine.iloc[-1,3:-2] - manNmachine.iloc[-2,3:-2] 
px.bar(data_frame=Last_change,x=Last_change.index,y=Last_change.values)

[Back to Contents](#cont)

### <a id="vis_2"> How is the distribution of all equipments on last day?

In [12]:
#The last change
manmachine = manNmachine.columns[3:]
fire_col = go.Figure()
fire_col.add_trace(go.Scatter(x=manmachine,y=manNmachine.iloc[-1,3:-2]))

[Back to Contents](#cont)

In [13]:
pivot_mNm = manNmachine.iloc[:,2:]
pivot_mNm = pivot_mNm.T
pivot_mNm

,0,1,2,3,4,5
aircraft,27.0,29.0,29.0,30.0,30.0,33.0
helicopter,26.0,29.0,29.0,31.0,31.0,37.0
tank,146.0,150.0,198.0,211.0,217.0,251.0
APC,706.0,816.0,846.0,862.0,900.0,939.0
field artillery,49.0,74.0,77.0,85.0,90.0,105.0
BUK,1.0,1.0,1.0,1.0,1.0,1.0
MRL,4.0,21.0,24.0,40.0,42.0,50.0
military auto,30.0,291.0,305.0,355.0,374.0,404.0
fuel tank,60.0,60.0,60.0,60.0,60.0,60.0
drone,2.0,3.0,3.0,3.0,3.0,3.0


In [14]:
#Where the equipments are??
pivot_mNm.loc[:2,'type'] = 'Air'
pivot_mNm.loc[2:9,'type'] = 'Land'
pivot_mNm.loc['anti-aircraft warfare','type'] = 'Land'
pivot_mNm.loc['personnel','type'] = 'Land'
pivot_mNm.loc['drone','type'] = 'Air'
pivot_mNm.loc['naval ship','type'] = 'Sea'

### <a id="vis_1"> Where is the action happening?

In [15]:
theatre = pivot_mNm.groupby('type')[5].sum().reset_index()
px.bar(data_frame=theatre,x='type',y=5)

[Back to Contents](#cont)